google job search scraping tuto : https://serpapi.com/blog/scrape-google-jobs-organic-results-with-python/


In [1]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
package_path = '/content/packages'
os.symlink('/content/drive/MyDrive/Packages', package_path)
sys.path.insert(0, package_path)

Mounted at /content/drive


## IMPORT NECESSARY LIBRARIES

In [2]:
import pandas as pd
import csv 

# Libraries for using google jobs API from SERPAPI
from serpapi import GoogleSearch

import json, os
import datetime

# Library to generate UULE code on the fly
import uule_grabber

## GET GEOTARGETS INTO UULE CODE

In [3]:
# assign all google geo location to dataset
geotargets = pd.read_csv('/content/drive/MyDrive/github/gg_job_search/src/geotargets-2022-12-21.csv') 

# get canonical name for our cities of interest >>> PARIS for instance
canonical_name = geotargets.loc[(geotargets['Target Type'] == 'Country') &( geotargets['Country Code'] == 'FR'), ['Canonical Name']]
canonical_name = canonical_name.iloc[0,0]

# get corresponding uule code to cities of interest
cities = canonical_name
uule_codes = uule_grabber.uule(cities)

## COLLECT DATA W\ SERPAPI

In [4]:
# Defining our search query + necessary parameters for GoogleSearch object

search_queries = ["machine learning engineer", "data scientist", "data analyst", "data engineer"]

# initialize jobs_all outside of the loop
jobs_all = pd.DataFrame()

# initialize all_jobs_queries outside of function
all_jobs_queries = pd.DataFrame()

#for date in datelist:
for query in search_queries:

    # serpapi will iterate up to n number of iterations
    for num  in range(45):

        start = num * 10


    # define parameters
        params = {
            'api_key': '4b799b64af09be918f6d66d6e908184cba836c46596e58bfa8bf1fb9280e7f09',                              
            'device':'desktop',
            'uule': uule_codes,                         # encoded location 
            'q': query,                          # search query
            'google_domain': 'google.fr',              
            'hl': 'fr',                                 # language of the search
            'gl': 'fr',                                 # country of the search
            'engine': 'google_jobs',                    # SerpApi search engine
            'start': start,                             # pagination
            'chips': 'date_posted:today'  #'data_range:2023-02-03'   #'date_posted:today'                 
        }

        # get results 
        search = GoogleSearch(params)
        results = search.get_dict()  # JSON file to python dict 

        # check if last search page, exceptions handling   
        try:
            if results['error'] == "Google hasn't returned any results for this query.":
                    break
        except KeyError:
                print(f"Getting SerpAPI data for page: {start} of '{query}' results")
        else:
                continue

        # create dataframe of 10 pulled results
        jobs = results['jobs_results']
        jobs = pd.DataFrame(jobs)
        jobs = pd.concat([pd.DataFrame(jobs), 
                        pd.json_normalize(jobs['detected_extensions'])], #convert detected extension key in json files into pandas df
                        axis=1).drop('detected_extensions', axis=1) # drop json object
        jobs['date_time'] = datetime.datetime.now() # add extraction date column for job results

        # concat dataframe of 10 pulled results with jobs_all
        if start == 0:
                jobs_all = jobs
        else:
                jobs_all = pd.concat([jobs_all, jobs])

        # assign ongoing query to pulled results dataframe
        jobs_all['search_query'] = query

        # concat dataframe of all pulled results with all_jobs_queries 
        all_jobs_queries = pd.concat([all_jobs_queries, jobs_all])

# get rid of duplicates before exportation
all_jobs_queries.drop_duplicates(subset='description', inplace=True)

# reindex columns to match the order of existing data
all_jobs_queries = all_jobs_queries.reindex(columns=['title', 'company_name', 'location', 'via', 'description',
       'job_highlights', 'related_links', 'thumbnail', 'extensions', 'job_id',
       'posted_at', 'schedule_type', 'date_time', 'search_query'])

all_jobs_queries.to_csv('/content/drive/MyDrive/github/gg_job_search/data/gg_job_search_all_RAW.csv', mode='a', header=False, index=False)

print("Shape of df:", all_jobs_queries.shape)

Shape of df: (0, 14)
